In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
PATH = 'data/'
!ls {PATH}

application_test.csv	     HomeCredit_columns_description.csv
application_test.csv.zip     installments_payments.csv
application_train.csv	     installments_payments.csv.zip
application_train.csv.zip    POS_CASH_balance.csv
bureau_balance.csv	     POS_CASH_balance.csv.zip
bureau_balance.csv.zip	     previous_application.csv
bureau.csv		     previous_application.csv.zip
bureau.csv.zip		     sample_submission.csv
credit_card_balance.csv      sample_submission.csv.zip
credit_card_balance.csv.zip


In [8]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

In [6]:
df_app = pd.read_csv(f'{PATH}application_train.csv', low_memory=False)

In [16]:
display_allRows(df_app.transpose())

,0,1,2,3,4,...,307506,307507,307508,307509,307510
SK_ID_CURR,100002,100003,100004,100006,100007,...,456251,456252,456253,456254,456255
TARGET,1,0,0,0,0,...,0,0,0,1,0
NAME_CONTRACT_TYPE,Cash loans,Cash loans,Revolving loans,Cash loans,Cash loans,...,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans
CODE_GENDER,M,F,M,F,M,...,M,F,F,F,F
FLAG_OWN_CAR,N,N,Y,N,N,...,N,N,N,N,N
FLAG_OWN_REALTY,Y,N,Y,Y,Y,...,N,Y,Y,Y,N
CNT_CHILDREN,0,0,0,0,0,...,0,0,0,0,0
AMT_INCOME_TOTAL,202500,270000,67500,135000,121500,...,157500,72000,153000,171000,157500
AMT_CREDIT,406598,1.2935e+06,135000,312682,513000,...,254700,269550,677664,370107,675000
AMT_ANNUITY,24700.5,35698.5,6750,29686.5,21865.5,...,27558,12001.5,29979,20205,49117.5


In [5]:
df_bru = pd.read_csv(f'{PATH}bureau.csv')

In [21]:
display_allRows(df_bru.transpose())

,0,1,2,3,4,...,1716423,1716424,1716425,1716426,1716427
SK_ID_CURR,215354,215354,215354,215354,215354,...,259355,100044,100044,246829,246829
SK_ID_BUREAU,5714462,5714463,5714464,5714465,5714466,...,5057750,5057754,5057762,5057770,5057778
CREDIT_ACTIVE,Closed,Active,Active,Active,Active,...,Active,Closed,Closed,Closed,Closed
CREDIT_CURRENCY,currency 1,currency 1,currency 1,currency 1,currency 1,...,currency 1,currency 1,currency 1,currency 1,currency 1
DAYS_CREDIT,-497,-208,-203,-203,-629,...,-44,-2648,-1809,-1878,-463
CREDIT_DAY_OVERDUE,0,0,0,0,0,...,0,0,0,0,0
DAYS_CREDIT_ENDDATE,-153,1075,528,NaN,1197,...,-30,-2433,-1628,-1513,NaN
DAYS_ENDDATE_FACT,-153,NaN,NaN,NaN,NaN,...,NaN,-2493,-970,-1513,-387
AMT_CREDIT_MAX_OVERDUE,NaN,NaN,NaN,NaN,77674.5,...,0,5476.5,NaN,NaN,NaN
CNT_CREDIT_PROLONG,0,0,0,0,0,...,0,0,0,0,0


In [6]:
df_bruBl = pd.read_csv(f'{PATH}bureau_balance.csv')

In [10]:
display_all(df_bruBl.transpose())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,...,27299425,27299426,27299427,27299428,27299429,27299430,27299431,27299432,27299433,27299434,27299435,27299436,27299437,27299438,27299439,27299440,27299441,27299442,27299443,27299444,27299445,27299446,27299447,27299448,27299449,27299450,27299451,27299452,27299453,27299454,27299455,27299456,27299457,27299458,27299459,27299460,27299461,27299462,27299463,27299464,27299465,27299466,27299467,27299468,27299469,27299470,27299471,27299472,27299473,27299474,27299475,27299476,27299477,27299478,27299479,27299480,27299481,27299482,27299483,27299484,27299485,27299486,27299487,27299488,27299489,27299490,27299491,27299492,27299493,27299494,27299495,27299496,27299497,27299498,27299499,27299500,27299501,27299502,27299503,27299504,27299505,27299506,27299507,27299508,27299509,27299510,27299511,27299512,27299513,27299514,27299515,27299516,27299517,27299518,27299519,27299520,27299521,27299522,27299523,27299524,27299525,27299526,27299527,27299528,27299529,27299530,27299531,27299532,27299533,27299534,27299535,27299536,27299537,27299538,27299539,27299540,27299541,27299542,27299543,27299544,27299545,27299546,27299547,27299548,27299549,27299550,27299551,27299552,27299553,27299554,27299555,27299556,27299557,27299558,27299559,27299560,27299561,27299562,27299563,27299564,27299565,27299566,27299567,27299568,27299569,27299570,27299571,27299572,27299573,27299574,27299575,27299576,27299577,27299578,27299579,27299580,27299581,27299582,27299583,27299584,27299585,27299586,27299587,27299588,27299589,27299590,27299591,27299592,27299593,27299594,27299595,27299596,27299597,27299598,27299599,27299600,27299601,27299602,27299603,27299604,27299605,27299606,27299607,27299608,27299609,27299610,27299611,27299612,27299613,27299614,27299615,27299616,27299617,27299618,27299619,27299620,27299621,27299622,27299623,27299624,27299625,27299626,27299627,27299628,27299629,27299630,27299631,27299632,27299633,27299634,27299635,27299636,27299637,27299638,27299639,27299640,27299641,27299642,27299643,27299644,27299645,27299646,27299647,27299648,27299649,27299650,27299651,27299652,27299653,27299654,27299655,27299656,27299657,27299658

In [7]:
df_bruAll = pd.merge(df_bruBl,df_bru, on='SK_ID_BUREAU', how='right')

In [9]:
display_all(df_bruAll)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,5715448,0.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
1,5715448,-1.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
2,5715448,-2.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
3,5715448,-3.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
4,5715448,-4.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
5,5715448,-5.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
6,5715448,-6.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
7,5715448,-7.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
8,5715448,-8.0,C,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000
9,5715448,-9.0,0,380361,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.000,0.000,67500.000,0.0,Credit card,-183,0.000


In [25]:
df_ccdBl = pd.read_csv(f'{PATH}credit_card_balance.csv')
display_allRows(df_ccdBl.transpose())

,0,1,2,3,4,...,3840307,3840308,3840309,3840310,3840311
SK_ID_PREV,2562384,2582071,1740877,1389973,1891521,...,1036507,1714892,1302323,1624872,2411345
SK_ID_CURR,378907,363914,371185,337855,126868,...,328243,347207,215757,430337,236760
MONTHS_BALANCE,-6,-1,-7,-4,-1,...,-9,-9,-9,-10,-10
AMT_BALANCE,56.97,63975.6,31815.2,236572,453919,...,0,0,275785,0,0
AMT_CREDIT_LIMIT_ACTUAL,135000,45000,450000,225000,450000,...,45000,45000,585000,450000,157500
AMT_DRAWINGS_ATM_CURRENT,0,2250,0,2250,0,...,NaN,0,270000,NaN,0
AMT_DRAWINGS_CURRENT,877.5,2250,0,2250,11547,...,0,0,270000,0,0
AMT_DRAWINGS_OTHER_CURRENT,0,0,0,0,0,...,NaN,0,0,NaN,0
AMT_DRAWINGS_POS_CURRENT,877.5,0,0,0,11547,...,NaN,0,0,NaN,0
AMT_INST_MIN_REGULARITY,1700.33,2250,2250,11795.8,22924.9,...,0,0,2250,0,0


In [33]:
df_instpay = pd.read_csv(f'{PATH}installments_payments.csv')
display_allRows(df_instpay.transpose())

,0,1,2,3,4,...,13605396,13605397,13605398,13605399,13605400
SK_ID_PREV,1054186.00,1330831.000,2085231.0,2452527.00,2714724.000,...,2186857.0,1310347.0,1308766.000,1062206.0,2448869.00
SK_ID_CURR,161674.00,151639.000,193053.0,199697.00,167756.000,...,428057.0,414406.0,402199.000,409297.0,434321.00
NUM_INSTALMENT_VERSION,1.00,0.000,2.0,1.00,1.000,...,0.0,0.0,0.000,0.0,1.00
NUM_INSTALMENT_NUMBER,6.00,34.000,1.0,3.00,2.000,...,66.0,47.0,43.000,43.0,19.00
DAYS_INSTALMENT,-1180.00,-2156.000,-63.0,-2418.00,-1383.000,...,-1624.0,-1539.0,-7.000,-1986.0,-27.00
DAYS_ENTRY_PAYMENT,-1187.00,-2156.000,-63.0,-2426.00,-1366.000,...,NaN,NaN,NaN,NaN,NaN
AMT_INSTALMENT,6948.36,1716.525,25425.0,24350.13,2165.040,...,67.5,67.5,43737.435,67.5,11504.25
AMT_PAYMENT,6948.36,1716.525,25425.0,24350.13,2160.585,...,NaN,NaN,NaN,NaN,NaN


In [34]:
df_posCb = pd.read_csv(f'{PATH}POS_CASH_balance.csv')
display_allRows(df_posCb.transpose())

,0,1,2,3,4,...,10001353,10001354,10001355,10001356,10001357
SK_ID_PREV,1803195,1715348,1784872,1903291,2341044,...,2448283,1717234,1283126,1082516,1259607
SK_ID_CURR,182943,367990,397406,269225,334279,...,226558,141565,315695,450255,174278
MONTHS_BALANCE,-31,-33,-32,-35,-35,...,-20,-19,-21,-22,-52
CNT_INSTALMENT,48,36,12,48,36,...,6,12,10,12,16
CNT_INSTALMENT_FUTURE,45,35,9,42,35,...,0,0,0,0,0
NAME_CONTRACT_STATUS,Active,Active,Active,Active,Active,...,Active,Active,Active,Active,Completed
SK_DPD,0,0,0,0,0,...,843,602,609,614,0
SK_DPD_DEF,0,0,0,0,0,...,0,0,0,0,0


In [35]:
df_preApp = pd.read_csv(f'{PATH}previous_application.csv')
display_allRows(df_preApp.transpose())

,0,1,2,3,4,...,1670209,1670210,1670211,1670212,1670213
SK_ID_PREV,2030495,2802425,2523466,2819243,1784265,...,2300464,2357031,2659632,2785582,2418762
SK_ID_CURR,271877,108129,122040,176158,202054,...,352015,334635,249544,400317,261212
NAME_CONTRACT_TYPE,Consumer loans,Cash loans,Cash loans,Cash loans,Cash loans,...,Consumer loans,Consumer loans,Consumer loans,Cash loans,Cash loans
AMT_ANNUITY,1730.43,25188.6,15060.7,47041.3,31924.4,...,14704.3,6622.02,11520.9,18821.5,16431.3
AMT_APPLICATION,17145,607500,112500,450000,337500,...,267296,87750,105237,180000,360000
AMT_CREDIT,17145,679671,136444,470790,404055,...,311400,64291.5,102524,191880,360000
AMT_DOWN_PAYMENT,0,NaN,NaN,NaN,NaN,...,0,29250,10525.5,NaN,NaN
AMT_GOODS_PRICE,17145,607500,112500,450000,337500,...,267296,87750,105237,180000,360000
WEEKDAY_APPR_PROCESS_START,SATURDAY,THURSDAY,TUESDAY,MONDAY,THURSDAY,...,WEDNESDAY,TUESDAY,MONDAY,WEDNESDAY,SUNDAY
HOUR_APPR_PROCESS_START,15,11,11,7,9,...,12,15,12,9,10
